In [ ]:
import google.generativeai as genai
import os

In [ ]:
genai.configure(api_key="Enter-your-api-key")
model = genai.GenerativeModel("gemini-1.5-flash")

# Model & It's Function


In [ ]:
from tensorflow.keras.models import load_model
from PIL import Image, ImageTk as PILImageTk
import speech_recognition as sr
import customtkinter as ctk
import numpy as np
import threading
import sys
import pyttsx3
import queue
import time
import cv2
import re

In [ ]:
model_currency = load_model('model/model_1.h5')

In [ ]:
Classes = {0: "10", 1: "10", 2: "100", 3: "20", 4: "20", 5: "200", 6:"5", 7:"50"}  

In [ ]:
def predict_class(model, image):
    
    image = cv2.resize(image, (224, 224))
       
    
    # Reshape the image to match the input shape expected by the model (add batch dimension)
    image_expanded = np.expand_dims(image, axis=0)

    # Predict the image using the loaded model
    prediction = model.predict(image_expanded)

    # Get the predicted label (class index with the highest probability)
    predicted_label = np.argmax(prediction)

    
    return predicted_label

In [ ]:
def capture_one_face():
    
    if cap.isOpened():
        _, frame = cap.read()
        
        text_to_speech("Photo captured successfully. please wait..")
        
        # Assuming predict_class returns an index corresponding to the Classes dictionary
        currency_index = predict_class(model_currency, frame)
        
        # Debugging the result
        print(f"Predicted index: {currency_index}")
        
        # Check if the index is valid
        if currency_index in Classes:
            currency = Classes[currency_index]
            text = f"found {currency} Egyptian Pound"
            text_to_speech(text)
        return currency
            

# Commands

## 1- list commands

## 2- Extract total amount

## 3- list items

## 4- check balance / amount 

## 5- payment

# Features

In [ ]:
def Extract_Total(image_path):
    
    myfile = genai.upload_file(image_path)
    
    
    total_amount = model.generate_content(
        
        [myfile, "\n\n",  ''' Extract total from the image  in the json schema:

             ''']
    )
    
   
    # Start the chat with initial history
    history = [
        {"role": "user", "parts": "Hello"},
        {"role": "model", "parts": "Great to meet you. What would you like to know?"}
    ]

    history.append({"role": "user", "parts": f"Total amount Extract from invoice in json format is {total_amount.text} "})
    history.append({"role": "model", "parts": f"That's great! ok"})

    # Start chat with updated history
    chat = model.start_chat(history=history)

    # Now, you can send the follow-up question about the number of paws
    response = chat.send_message("what is total amount of invoice? i want only number")
    
    return response.text



In [ ]:
def Extract_items(image_path):
    
    
    myfile = genai.upload_file(image_path)
    
    
    
    
    items = model.generate_content(
    [myfile, "\n\n",  ''' Extract QTY,DESCRIPTION,UNIT PRICE from the image  in the json schema: 

         ''']
    )
    
    
   

    # Start the chat with initial history
    history = [
        {"role": "user", "parts": "Hello"},
        {"role": "model", "parts": "Great to meet you. What would you like to know?"}
    ]
    
    history.append({"role": "user", "parts": f"all element of invoice in json format is {items.text} "})
    
    chat = model.start_chat(history=history)
    
    
    response = chat.send_message(
      "Can you provide a list of only the items I ordered, including their quantities and totals? I want it in this format: You ordered the following items:\n\n* Item Name - Quantity: X, Total: Y"
    )

    
    
    return response.text


In [ ]:
def check_invoice(image_path):
    
    myfile = genai.upload_file(image_path)
    
    
    
    
    check = model.generate_content(
    [myfile, "\n\n",  ''' is this invoice ? answer with yes or no

         ''']
    )
  
    
    return check.text


In [ ]:
def check_clear(image_path):
    
    myfile = genai.upload_file(image_path)
    
    
    
    
    check = model.generate_content(
    [myfile, "\n\n",  ''' is this invoice clear to extract information ? answer with yes or no

         ''']
    )
  
    
    return check.text

In [ ]:
def start():
     check = check_invoice("captured_image.jpg")
     if "yes" in check.lower():
         flag = check_clear("captured_image.jpg")
         if "yes" in flag.lower():
             return 1
         else:
             return 0
     else:
        return -1

# GUI (MAIN FUNCTIONS)

In [ ]:
import customtkinter as ctk
import speech_recognition as sr
from PIL import Image
import threading
import sys
import pyttsx3
import queue
import time
import cv2
import re
from CTkMessagebox import CTkMessagebox
import customtkinter as ctk


ff_sound = 0
# Flag to indicate when to stop the background thread
stop_listening = False

# Queue for thread-safe communication
gui_queue = queue.Queue()

# Initialize text-to-speech engine
engine = pyttsx3.init()


imgtk = None

total_amount = 0

def text_to_speech(text):
    try:
        engine.setProperty('rate', 170)
        engine.say(text)
        engine.runAndWait()
    except RuntimeError:
        # If the engine is already running, just pass
        pass
    
def welcome():
    
    
    text_to_speech("Welcome to SightPay! Use voice commands to interact with the app.for more information say help")
    
    


    
    



# Function to update the camera frame in the UI
def update_frame():
    global imgtk  # Retain a reference to prevent garbage collection
    if cap.isOpened():
        _, frame = cap.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert the frame to RGB
        img = Image.fromarray(frame)
        imgtk = ctk.CTkImage(img, size=(w-200, h-100))  # Use CTkImage instead of PILImageTk.PhotoImage
        camera_label.imgtk = imgtk  # Store reference to avoid being garbage collected
        camera_label.configure(image=imgtk)  # Update the label with the image
        camera_label.after(10, update_frame)  # Refresh the frame every 10ms


# Function to perform text-to-speech
# def text_to_speech(text):
#     try:
#         engine.say(text)
#         engine.runAndWait()
#     except RuntimeError:
#         # If the engine is already running, just pass
#         pass

# Function to save a high-resolution image from the camera
def capture_image():
    
    if cap.isOpened():

        # Capture a frame after setting the resolution
        _, high_res_frame = cap.read()
            
        # Assuming predict_class returns an index corresponding to the Classes dictionary
        currency_index = predict_class(model_currency, high_res_frame)
        
        # Debugging the result
        print(f"Predicted index: {currency_index}")
        
        # Check if the index is valid
        if currency_index in Classes:
            currency = Classes[currency_index]
            text = f"found {currency} Egyptian Pound"
            text_to_speech(text)
            return currency
            
        
            
def capture_image_for_invoice():
    
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1920)  # Set high width (e.g., 1920 for 1080p)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1080)  # Set high height (e.g., 1080 for 1080p)
        
        # Capture a frame after setting the resolution
    _, high_res_frame = cap.read()
    cv2.imwrite("captured_image.jpg", high_res_frame)  # Save the current frame as an image
   
flag = 0 
flag_payment=0
flag_ocr=0
number=[]

flag_ur=0

# Function to recognize speech and display in the GUI
def recognize_speech():
    
        global flag_ur
        if flag_ur==0:
            time.sleep(2)
            welcome()
            flag_ur=1
            
        
      
        global stop_listening
        global total_amount
        global flag
        global flag_payment
        recognizer = sr.Recognizer()

        with sr.Microphone() as source:
            try:
                if stop_listening:
                    return  # Stop the function if the flag is set to True

                audio = recognizer.listen(source)
                text = recognizer.recognize_google(audio)

                text = text.lower()
                print("main")
                print(text)

                # Use regular expressions to match exact phrases
                if re.search(r"\bExtract  total amount\b", text) or re.search(r"\btotal amount\b", text) or re.search(r"\bExtract total\b", text) and flag==0:


                    text_to_speech("OK,Capturing a photo ..")
                    capture_image_for_invoice()
                    text_to_speech("Photo captured successfully. Please hold on while we process and read the invoice")
                    f = start()
                    if f==1:
                        total = Extract_Total("captured_image.jpg")
                        text_to_speech(total)
                    elif f==-1:
                        text_to_speech("Sorry , this is not invoice")
                    else:
                        text_to_speech("Sorry , the information not clear please try again")



                elif re.search(r"\bhelp\b", text) or re.search(r"\bcan you help me\b", text) and flag==0:  

                    text_to_speech("Welcome to SightPay! SightPay is an accessible payment app designed for seamless transactions using voice commands")


                    commands = [
                        "Say 'Payment' to start a transaction. The system will first scan the invoice, if available, and then count the cash.",
                        "Say 'Check balance' to count the available cash ",
                        "Say 'Total Invoice Amount' to extract and announce the total amount from the invoice.",
                        "Say 'List Invoice Items' to list all items mentioned in the invoice.",
                        "Say 'Check Invoice' to verify whether the current document is a valid invoice.",
                        "Say 'Help' to hear a list of available commands and get usage instructions."
                    ]


                    command_text = "Here are the available commands: " + ", ".join(commands) + "."
                    text_to_speech(command_text)


                elif re.search(r"\blist items\b", text) or re.search(r"\bwhat items in invoice\b", text) and flag==0:

                    text_to_speech("OK,Capturing a photo ..")
                    capture_image()
                    text_to_speech("Photo captured successfully. Please hold on while we process and read the invoice")

                    f = start()
                    if f==1:
                        items = Extract_items("captured_image.jpg")
                        text_to_speech(items)
                    elif f==-1:
                        text_to_speech("Sorry , this is not invoice")
                    else:
                        text_to_speech("Sorry , the information not clear please try again")



                elif re.search(r"\bcheck invoice\b", text) and flag==0:

                    text_to_speech("OK,Capturing a photo ..")
                    capture_image()
                    text_to_speech("Photo captured successfully. Please hold on while we process and check invoice")
                    check = check_invoice("captured_image.jpg")
                    if "yes" in check.lower():
                             text_to_speech("yes this is invoice")
                             flag = check_clear("captured_image.jpg")
                             if "yes" in flag.lower():
                                text_to_speech("and information is clear")
                             else:
                                text_to_speech("and information not clear")                

                    else:
                             text_to_speech("no  this is not invoice")

                elif re.search(r"\bpayment\b", text) and flag==0:
                    flag=1
                    flag_payment=1
                    text_to_speech("Alright, the system will now start Transaction. first say read invoice To get the total amount,after that simply say 'take a photo'. When you're done, say 'end transaction' to finish the counting total amount.")

                elif re.search(r"\bcheck balance\b", text) and flag==0:

                    flag=1

                    text_to_speech("Alright, the system will now start counting the amount. To display the total, simply say 'take a photo'. When you're done, say 'end transaction' to finish the process.")



                if flag:
                    global number
                    global flag_ocr



                    if flag_payment:

                        if flag_ocr==0:
                             if re.search(r"\bread invoice\b", text):
                                 text_to_speech("reading Invoice")
                                 capture_image_for_invoice()
                                 text_to_speech("Invoice captured successfully please wait")
                                 total = Extract_Total("captured_image.jpg")
                                 print(total)
                                 items = re.findall(r'\d+', total)
                                 print(items)
                                 if items:
                                        flag_ocr=1
                                        text_to_speech(f"total amount is {items[0]} ")
                                        text_to_speech("now we can count Money")
                                        number.append(items[0])
                                 else:
                                     text_to_speech("try again total can't be extracted")

                        if flag_ocr==1:

                            if re.search(r"\btake a photo\b", text):
                                text_to_speech("OK, Capturing a photo ..")
                                total_amount += int(capture_one_face())

                            elif re.search(r"\bend transaction\b", text):
                                text_to_speech("Ending Transaction") 
                                text_to_speech(f"invoice total amount is {number[0]}")
                                text_to_speech(f"total amount is {total_amount} Egyption pound")
                                if int(number[0])>int(total_amount):
                                    text_to_speech("sorry your balance less than required for paying invoice")
                                else:
                                    text_to_speech(f"Transaction completed successfully Reminder is {number[0]-total_amount} Egyption pound. ")

                                total_amount=0
                                flag=0
                                flag_payment=0
                                flag_ocr=0
                                number.clear()



                    else:   
                            if re.search(r"\btake a photo\b", text):
                                text_to_speech("OK, Capturing a photo ..")
                                total_amount += int(capture_one_face())

                            elif re.search(r"\bend transaction\b", text):
                                text_to_speech("Ending Transaction") 
                                text_to_speech(f"total amount is {total_amount} Egyption pound")
                                total_amount=0
                                flag=0









            except sr.UnknownValueError:
                pass

            except sr.RequestError:
                text_to_speech("Could not request results; check your network connection.")
            except OSError as e:
                text_to_speech(f"An error occurred with the audio stream: {str(e)}")

            except Exception :
                pass


            # Restart listening automatically after each recognition
            if not stop_listening:
                recognize_speech_in_background()



# Run speech recognition in a separate thread
def recognize_speech_in_background():
    
        threading.Thread(target=recognize_speech, daemon=True).start()
    
    



# Handle app closing event
def on_closing():
    global stop_listening
    global running_balance_check
    cap.release()
    stop_listening = True  # Set the flag to stop background threads
    app.destroy()          # Close the GUI application


# Function to update GUI from the main thread
def update_gui():
    try:
        while True:
            message = gui_queue.get_nowait()
            if message[0] == "status":
                status_label.configure(text=message[1])
            gui_queue.task_done()
    except queue.Empty:
        pass
    finally:
        app.after(10, update_gui)  # Schedule the next update


# Create a simple GUI using customtkinter
app = ctk.CTk()

ctk.set_appearance_mode("dark")

screen_width = app.winfo_screenwidth()
screen_height = app.winfo_screenheight()
w = 720
h = 750
a = (screen_width // 2) - (w // 2)
b = (screen_height // 2) - (h // 2)
app.geometry(f"{w}x{h}+{a}+{b}")
app.title("SightPay")


camera_frame = ctk.CTkFrame(app, width=w, height=h-100)  # Subtract some space for the button

camera_frame.pack(pady=20)

# Create a label to display the camera feed with no text
camera_label = ctk.CTkLabel(camera_frame, width=w, height=h-100, text="")
camera_label.pack(fill="both", expand=True)  # Ensure the label fills the frame


# Access the laptop camera
cap = cv2.VideoCapture(0)

# Check if camera opened successfully
if not cap.isOpened():
    show_custom_messagebox("Error", "Unable to access camera")
else:
    # Set the default resolution for the camera feed
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

    # Start updating the camera frame
    update_frame()



# Start voice recognition automatically when the app launches



recognize_speech_in_background()



# Start the GUI update loop
update_gui()


# Bind the close event to on_closing function
app.protocol("WM_DELETE_WINDOW", on_closing)

# Run the application
app.mainloop()
